In [1]:
import altair as alt
from vega_datasets import data
from jenkspy import JenksNaturalBreaks # sinngle dimensional K-means to find cohorts or "natura breaks" for discrete labeliing

import pandas as pd
alt.renderers.enable("html")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df = pd.read_csv("../county-plot.csv")
df["uses_competitor"] = df["uses_competitor"].fillna("")
df["id"] = df["geo_id"].apply(lambda s: int(s.split("US")[1]))
# df = df.melt(id_vars=["id","geo_id","total_population_5+","geo_name","uses_competitor"])


In [4]:
counties = alt.topo_feature(data.us_10m.url, 'counties')

def make_choropleth(color_col: str, title: str, selection,  color_scheme: str = "inferno", scaletype: str = "jnb", height=600,width=800) -> alt.Chart: 
    if scaletype == "jnb":
        jnb = JenksNaturalBreaks(5)
        jnb.fit(df[color_col])
        scale = alt.Scale(type='quantile',scheme=color_scheme,domain=jnb.inner_breaks_)
    else: 
        scale = alt.Scale(type='linear')
    
    choropleth = alt.Chart(counties, title=title).mark_geoshape().transform_lookup(
        lookup="id", from_=alt.LookupData(data=df, key="id", fields=["id","geo_id","geo_name",color_col])
    ).encode(
        color=alt.Color(
            f"{color_col}:Q",
            legend=alt.Legend(direction="horizontal", orient="bottom"),
            scale=scale,
        ),
        tooltip=["geo_name:N",f"{color_col}:Q"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    ).project(
        type="albersUsa"
    ).properties(
        height=900, width=1200
    )
    return choropleth

In [7]:



selection = alt.selection_point(fields=['geo_id'])
# selection = alt.selection_interval()
charts = [
    make_choropleth("pct_pop_below_poverty_level", "Percent of US Population below the Poverty Level by County",selection,"teals"),
    make_choropleth("pct_speaks_other_than_english","Percent of US population speaking a language other than english",selection,"blues"),
    make_choropleth("pct_households_receiving_food_stamps","Percent of population recieving SNAP benefits",selection,"greens")
]

multi_choropleth = alt.vconcat(*charts).resolve_scale(
    color='independent'
).add_params(selection)

multi_choropleth


alt.VConcatChart(...)

In [ ]:
c = multi_choropleth.to_json()
with open("../outputs/multi-choropleth.json","w") as fp: # for online hostings
    print(c,file=fp)